In [1]:
import sys
sys.path.append("../")

In [2]:
import logging
import os
import threadpoolctl
import torch
from pathlib import Path
from lightning_fabric import seed_everything

import numpy as np
import pandas as pd
from rectools import Columns

from implicit.als import AlternatingLeastSquares

from rectools.dataset import Dataset
from rectools.metrics import MAP, calc_metrics, MeanInvUserFreq, Serendipity
from rectools.models import ImplicitALSWrapperModel
from rectools.models import SASRecModel
from rectools.models.nn.item_net import CatFeaturesItemNet, IdEmbeddingsItemNet

In [3]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# For implicit ALS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger()

# Data

In [4]:
# %%time
# !wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
# !unzip -o data_original.zip
# !rm data_original.zip

In [5]:
DATA_PATH = Path("data_original")

interactions = (
    pd.read_csv(DATA_PATH / 'interactions.csv', parse_dates=["last_watch_dt"])
    .rename(columns={"last_watch_dt": "datetime"})
)

# Split dataset

In [6]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

# Split to train / test
max_date = interactions[Columns.Datetime].max()
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()
train.drop(train.query("total_dur < 300").index, inplace=True)

# drop items with less than 20 interactions in train
items = train["item_id"].value_counts()
items = items[items >= 20]
items = items.index.to_list()
train = train[train["item_id"].isin(items)]
    
# drop users with less than 2 interactions in train
users = train["user_id"].value_counts()
users = users[users >= 2]
users = users.index.to_list()
train = train[(train["user_id"].isin(users))]

users = train["user_id"].drop_duplicates().to_list()

# drop cold users from test
test_users_sasrec = test[Columns.User].unique()
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
test_users = test[Columns.User].unique()


In [7]:
items = pd.read_csv(DATA_PATH / 'items.csv')

In [8]:
# Process item features to the form of a flatten dataframe
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"

items["director"] = items["directors"].str.lower().replace(", ", ",", regex=False).str.split(",")
directors_feature = items[["item_id", "director"]].explode("director")
directors_feature.columns = ["id", "value"]
directors_feature["feature"] = "director"

content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
item_features_genre_content = pd.concat((genre_feature, content_feature))
item_features_genre_director = pd.concat((genre_feature, directors_feature))

candidate_items = interactions['item_id'].drop_duplicates().astype(int)
test["user_id"] = test["user_id"].astype(int)
test["item_id"] = test["item_id"].astype(int)

catalog=train[Columns.Item].unique()

In [9]:
dataset_no_features = Dataset.construct(
    interactions_df=train,
)

dataset_item_features = Dataset.construct(
    interactions_df=train,
    item_features_df=item_features_genre_content,
    cat_item_features=["genre", "content_type"],
)

dataset_item_features_genre_director = Dataset.construct(
    interactions_df=train,
    item_features_df=item_features_genre_director,
    cat_item_features=["genre", "director"],
)

In [10]:
metrics_name = {
    'MAP': MAP,
    'MIUF': MeanInvUserFreq,
    'Serendipity': Serendipity
    

}
metrics = {}
for metric_name, metric in metrics_name.items():
    for k in (1, 5, 10):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

# list with metrics results of all models
features_results = []

# SASRec

In [11]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

## Softmax loss

In [12]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [13]:
%%time
model.fit(dataset_item_features_genre_director)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params | Mode 
-----------------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 2.2 M  | train
-----------------------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.991     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode
/home/ma

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 7min 41s, sys: 19.3 s, total: 8min
Wall time: 7min 50s


In [14]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:322: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 2min 43s, sys: 8.04 s, total: 2min 51s
Wall time: 25 s


In [15]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "softmax"
features_results.append(metric_values)


In [16]:
features_results

[{'MAP@1': 0.04846577699474078,
  'MAP@5': 0.0816953145406517,
  'MAP@10': 0.09070442769366964,
  'MIUF@1': 3.871426206344739,
  'MIUF@5': 4.573068555853547,
  'MIUF@10': 5.159742458558834,
  'Serendipity@1': 0.001116687417059873,
  'Serendipity@5': 0.0008645696959881002,
  'Serendipity@10': 0.0007632648657992071,
  'model': 'softmax'}]

In [17]:
recos

,user_id,item_id,score,rank
0,73446,9728,2.401881,1
1,73446,7793,1.923069,2
2,73446,3784,1.824613,3
3,73446,3182,1.666528,4
4,73446,7829,1.662176,5
...,...,...,...,...
947045,857162,12995,2.385432,6
947046,857162,6809,2.360935,7
947047,857162,657,1.940931,8
947048,857162,4702,1.866479,9


## BCE loss

In [19]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

In [20]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    loss="BCE",
    n_negatives=2,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [21]:
%%time
model.fit(dataset_no_features)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params | Mode 
-----------------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 2.2 M  | train
-----------------------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.991     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode
/home/ma

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 7min 41s, sys: 18 s, total: 7min 59s
Wall time: 7min 51s


In [22]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn",
)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:322: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 2min 42s, sys: 6.58 s, total: 2min 48s
Wall time: 24.4 s


In [23]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "bce"
features_results.append(metric_values)

In [24]:
recos

,user_id,item_id,score,rank
0,73446,3182,3.370286,1
1,73446,12965,3.088001,2
2,73446,6774,3.056905,3
3,73446,16270,2.966968,4
4,73446,7582,2.965708,5
...,...,...,...,...
947045,857162,4151,2.733006,6
947046,857162,142,2.687315,7
947047,857162,9728,2.634741,8
947048,857162,3734,2.558933,9


## gBCE loss

In [26]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

In [27]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    loss="gBCE",
    n_negatives=256,
    gbce_t=0.75,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [28]:
%%time
model.fit(dataset_no_features)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params | Mode 
-----------------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 2.2 M  | train
-----------------------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.991     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode
/home/ma

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 2h 23min 5s, sys: 1min 12s, total: 2h 24min 18s
Wall time: 13min 45s


In [29]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)


/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:322: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 2min 55s, sys: 6.53 s, total: 3min 1s
Wall time: 23.3 s


In [30]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "gBCE"
features_results.append(metric_values)

## Softmax with key_padding_mask + causal_mask

In [32]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
    use_key_padding_mask=True,
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [33]:
%%time
model.fit(dataset_no_features)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params | Mode 
-----------------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 2.2 M  | train
-----------------------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.991     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode
/home/ma

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 7min 56s, sys: 15.9 s, total: 8min 12s
Wall time: 7min 53s


In [34]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:322: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 2min 51s, sys: 6.39 s, total: 2min 58s
Wall time: 23.2 s


In [35]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)

In [36]:
metric_values["model"] = "softmax_padding_mask"
features_results.append(metric_values)

In [37]:
features_df = (
    pd.DataFrame(features_results)
    .set_index("model")
    .sort_values(by=["MAP@10", "Serendipity@10"], ascending=False)
)
features_df

,MAP@1,MAP@5,MAP@10,MIUF@1,MIUF@5,MIUF@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,
softmax_padding_mask,0.049449,0.083399,0.092535,3.703682,4.303675,4.937142,0.000960,0.000727,0.000670
softmax,0.048466,0.081695,0.090704,3.871426,4.573069,5.159742,0.001117,0.000865,0.000763
gBCE,0.040848,0.072356,0.080166,2.332397,3.093763,3.942205,0.000103,0.000118,0.000134
bce,0.027035,0.051244,0.059080,3.882081,4.384314,4.734298,0.000104,0.000121,0.000131


### sasrec with item ids embeddings in ItemNetBlock

In [39]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [40]:
%%time
model.fit(dataset_no_features)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params | Mode 
-----------------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 2.2 M  | train
-----------------------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.991     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode
/home/ma

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 7min 45s, sys: 17.2 s, total: 8min 2s
Wall time: 7min 51s


In [41]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:322: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 2min 43s, sys: 5.55 s, total: 2min 49s
Wall time: 21.7 s


In [42]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_ids"
features_results.append(metric_values)

### sasrec with item ids and category features embeddings in ItemNetBlock

In [44]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, CatFeaturesItemNet),  # Use item ids and cat features in ItemNetBlock
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [45]:
#%%time
model.fit(dataset_item_features_genre_director)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params | Mode 
-----------------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 3.5 M  | train
-----------------------------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
13.903    Total estimated model params size (MB)
39        Modules in train mode
0         Modules in eval mode
/home/ma

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [46]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:322: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 2min 43s, sys: 6.03 s, total: 2min 49s
Wall time: 21 s


In [47]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_id_and_cat_features"
features_results.append(metric_values)

### sasrec with category item features embeddings in ItemNetBlock

In [49]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(CatFeaturesItemNet, ),  # Use only cat item features in ItemNetBlock
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [50]:
#%%time
model.fit(dataset_item_features_genre_director)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params | Mode 
-----------------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 2.0 M  | train
-----------------------------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.113     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode
/home/ma

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [51]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/nn/transformer_data_preparator.py:322: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/git_repos/RecTools/examples/../rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/maspirina1/git_repos/RecTools/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 4min 34s, sys: 5.32 s, total: 4min 39s
Wall time: 23 s


In [52]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_cat_features"
features_results.append(metric_values)

In [53]:
features_results

[{'MAP@1': 0.04846577699474078,
  'MAP@5': 0.0816953145406517,
  'MAP@10': 0.09070442769366964,
  'MIUF@1': 3.871426206344739,
  'MIUF@5': 4.573068555853547,
  'MIUF@10': 5.159742458558834,
  'Serendipity@1': 0.001116687417059873,
  'Serendipity@5': 0.0008645696959881002,
  'Serendipity@10': 0.0007632648657992071,
  'model': 'softmax'},
 {'MAP@1': 0.02703450310364319,
  'MAP@5': 0.05124396949349954,
  'MAP@10': 0.05907958022653049,
  'MIUF@1': 3.882081042459438,
  'MIUF@5': 4.384313936251787,
  'MIUF@10': 4.734298278984563,
  'Serendipity@1': 0.00010437879417622002,
  'Serendipity@5': 0.0001209341551851975,
  'Serendipity@10': 0.0001308852660453074,
  'model': 'bce'},
 {'MAP@1': 0.04084812884382748,
  'MAP@5': 0.07235604259743772,
  'MAP@10': 0.08016616686270196,
  'MIUF@1': 2.33239724771057,
  'MIUF@5': 3.093763291371006,
  'MIUF@10': 3.9422054591506033,
  'Serendipity@1': 0.00010303205538126172,
  'Serendipity@5': 0.00011795153034776448,
  'Serendipity@10': 0.00013442022189753792,
  

In [54]:
features_df = (
    pd.DataFrame(features_results)
    .set_index("model")
    .sort_values(by=["MAP@10", "Serendipity@10"], ascending=False)
)
features_df

,MAP@1,MAP@5,MAP@10,MIUF@1,MIUF@5,MIUF@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,
softmax_padding_mask,0.049449,0.083399,0.092535,3.703682,4.303675,4.937142,0.000960,0.000727,0.000670
sasrec_ids,0.048232,0.082287,0.091495,3.633264,4.308851,4.966090,0.001009,0.000803,0.000729
softmax,0.048466,0.081695,0.090704,3.871426,4.573069,5.159742,0.001117,0.000865,0.000763
sasrec_id_and_cat_features,0.046981,0.080645,0.090008,4.330347,4.974337,5.510450,0.001517,0.001109,0.000966
gBCE,0.040848,0.072356,0.080166,2.332397,3.093763,3.942205,0.000103,0.000118,0.000134
sasrec_cat_features,0.044048,0.071668,0.079515,4.009885,5.515872,6.006321,0.000966,0.000813,0.000715
bce,0.027035,0.051244,0.059080,3.882081,4.384314,4.734298,0.000104,0.000121,0.000131


### Item to item

In [55]:
target_items = [13865, 4457, 15297]

In [56]:
%%time
recos = model.recommend_to_items(
    target_items=target_items, 
    dataset=dataset_no_features,
    k=10,
    filter_itself=True,
    items_to_recommend=None, #white_list,
)

CPU times: user 10.3 s, sys: 600 ms, total: 10.9 s
Wall time: 1.68 s


In [57]:
recos

,target_item_id,item_id,score,rank
0,13865,11863,1.000000,1
1,13865,7107,1.000000,2
2,13865,6409,0.559578,3
3,13865,142,0.532108,4
4,13865,4457,0.531360,5
5,13865,6809,0.531058,6
6,13865,10440,0.522596,7
7,13865,2657,0.512829,8
8,13865,10772,0.511374,9
9,13865,15297,0.477553,10


In [58]:
# TODO: change model for recos (here is the last one trained and is is the worst in quality)
recos.merge(items[["item_id", "title"]], on="item_id")

,target_item_id,item_id,score,rank,title
0,13865,11863,1.000000,1,Девятаев - сериал
1,13865,7107,1.000000,2,Девятаев
2,13865,6409,0.559578,3,Особо опасен
3,13865,142,0.532108,4,Маша
4,13865,4457,0.531360,5,2067: Петля времени
5,13865,6809,0.531058,6,Дуров
6,13865,10440,0.522596,7,Хрустальный
7,13865,2657,0.512829,8,Подслушано
8,13865,10772,0.511374,9,Зелёная книга
9,13865,15297,0.477553,10,Клиника счастья
